# Homework 2 - Duc Trinh 

In [1]:
import pandas as pd

In [2]:
#BeautifulSoup stuff
from urllib.request import urlopen
import bs4 

In [3]:
import json

In [4]:
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

In [5]:
import pandas as pd

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import train_test_split

In [7]:
import numpy as np

In [ ]:
import sys
!{sys.executable} -m pip install graphviz
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin'

## Exercise 1 

### a)

In [ ]:
url = 'http://worldcup.sfg.io/matches'
data = urlopen(url).read().decode('utf-8')

In [ ]:
worldCup = json.loads(data)

In [ ]:
len(worldCup)

The number of matches played in the World Cup is the length of the list of dictionary we retrieved, which is 52.

### b)

In [ ]:
for match in worldCup:
    print(match['home_team_country'] + " - " + match['away_team_country'] + " " + str(match['home_team']['goals']) + ":" + str(match['away_team']['goals']))

### c)

In [ ]:
keys = ['fifa_id', 'location', 'datetime', 'home_team_country', 'away_team_country', 'winner', 'home_team_statistics', 'away_team_statistics']

In [ ]:
matches = [[match[i] for i in keys] for match in worldCup] 

In [ ]:
matchesFrame = pd.DataFrame(matches, columns= keys)
matchesFrame.head(5)

In [ ]:
matchesFrame["date"] = pd.DatetimeIndex(data = matchesFrame["datetime"]).date
temp = pd.Series(np.datetime_as_string(pd.DatetimeIndex(data = matchesFrame["datetime"]), unit = "m"))
matchesFrame["time"] =  "(" + temp.str[-5:-3].values + "," + temp.str[-2:].values + ")"
matchesFrame = matchesFrame.drop("datetime",axis=1)
matchesFrame.head(5)

Source cited for splicing numpy array of strings: https://stackoverflow.com/questions/39042214/how-can-i-slice-each-element-of-a-numpy-array-of-strings

### d)

In [ ]:
stadiumDict = {}
for loc in matchesFrame["location"]:
    if loc not in stadiumDict:
        stadiumDict[loc] = 1
    else:
        stadiumDict[loc] += 1
print(stadiumDict)

In [ ]:
stadiumFrame = pd.DataFrame.from_records(data = list(stadiumDict.items()), columns = ["City","Count"])
_ = stadiumFrame.plot.pie(y = "Count", figsize=(8,8), labels = stadiumFrame["City"], autopct=lambda x: '{:.0f}'.format(x * sum(stadiumFrame["Count"]) / 100), title = "Matches per stadium in the Women's Soccer World Cup", legend = 0, fontsize = 16)

Source used to show the wedge's value: https://stackoverflow.com/questions/14171021/matplotlib-pie-chart-how-to-replace-auto-labelled-relative-values-by-absolute-v

### e)

In [ ]:
keys2 = ['ball_possession', 'attempts_on_goal', 'pass_accuracy']

In [ ]:
teamData = [[homeTeam[i] for i in keys2] for homeTeam in list(matchesFrame["home_team_statistics"])]
teamData += [[awayTeam[i] for i in keys2] for awayTeam in list(matchesFrame["away_team_statistics"])] 

In [ ]:
teamFrame = pd.DataFrame(teamData, columns = keys2)
teamFrame = teamFrame.rename(columns = {'ball_possession':"Ball Possession", 'attempts_on_goal':'Attempts on Goal', 'pass_accuracy':'Pass Accuracy'})

In [ ]:
teamFrame.head(5)

In [ ]:
plt.figure(figsize = (8, 6))
plt.title(label = "Scatterplot of Ball Possession versus Attempts on Goal", fontsize = 14)
_ = sns.scatterplot(x='Ball Possession', y='Attempts on Goal', hue = 'Pass Accuracy', data=teamFrame)

From the plot, I can see a positive correlation between ball possession, attempts on goal for a team, and pass accuracy. This correlation makes sense since if a team is more skilled and has more control over the ball, they will have higher pass accuracy which would help them get a higher ball possession. More time with the ball and better pass accuracy will lead allow them more chances and time to attempt a goal which leads to an increases in attempts on goal.

## Exercise 2

### a)

In [ ]:
bank_data = pd.read_csv("../Data/bank.csv", delimiter = " ", names = ['age', 'sex', 'region', 'income', 'married', 'children', 'car','save_acct', 'current_acct', 'mortgage', 'pep'])
bank_data.head(5)

In [ ]:
numeric_data = bank_data.replace(['NO', 'YES', 'MALE', 'FEMALE'],[0,1,0,1])
numeric_data = numeric_data.values

features = np.zeros((len(numeric_data),4))
numeric_data = np.append(numeric_data,features,1)
j = 2
for i in range(len(numeric_data)):
    if numeric_data[i][j] == 'INNER_CITY':
        numeric_data[i][11:15] = [1,0,0,0]
    if numeric_data[i][j] == 'TOWN':
         numeric_data[i][11:15] = [0,1,0,0]
    if numeric_data[i][j] == 'RURAL':
         numeric_data[i][11:15] = [0,0,1,0]
    if numeric_data[i][j] == 'SUBURBAN':
         numeric_data[i][11:15] = [0,0,0,1]        
#remove redundant column
numeric_data = np.delete(numeric_data, 2, 1)
numeric_data.astype(float)

bank_labels = numeric_data[:, 9].astype(int)
bank_attrs  = np.delete(numeric_data, 9, 1)

In [ ]:
bank_features_train, bank_features_test, bank_labels_train, bank_labels_test = train_test_split(bank_attrs, bank_labels, test_size=0.33, random_state=42)

In [ ]:
decisionTree = DecisionTreeClassifier(max_depth = 6)
decisionTree.fit(bank_features_train, bank_labels_train)

In [ ]:
predictions_test = decisionTree.predict(bank_features_test)

In [ ]:
cm = metrics.confusion_matrix(bank_labels_test,predictions_test)
print(cm)

In [ ]:
print("Precision: ", metrics.precision_score(bank_labels_test,predictions_test))
print("Recall: ", metrics.recall_score(bank_labels_test,predictions_test))
print("Accuracy: ", metrics.accuracy_score(bank_labels_test,predictions_test))

### b) 

In [ ]:
tree1_prob = decisionTree.predict_proba(bank_features_test)
fpr, tpr, thresholds = metrics.roc_curve(bank_labels_test, tree1_prob[:,1], pos_label=1, sample_weight=None)

In [ ]:
metrics.roc_auc_score(bank_labels_test,tree1_prob[:,1], average='macro', sample_weight=None)

In [ ]:
decisionTree2 = DecisionTreeClassifier(max_depth = 10, splitter = "random")
decisionTree2.fit(bank_features_train, bank_labels_train)
tree2_prob = decisionTree2.predict_proba(bank_features_test)
fpr2, tpr2, thresholds2 = metrics.roc_curve(bank_labels_test, tree2_prob[:,1], pos_label=1, sample_weight=None)

In [ ]:
metrics.roc_auc_score(bank_labels_test,tree2_prob[:,1], average='macro', sample_weight=None)

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(fpr,tpr,'-r',linewidth=2.0, label = "First decision tree ROC curve")
straightLine = np.linspace(0,1,100)
plt.plot(straightLine, straightLine, '--k', linewidth=2.0, label = "y=x line")
plt.plot(fpr2,tpr2,'-b',linewidth=2.0, label = "Second decision tree ROC curve")
plt.xlabel('False positive rate', fontsize = 14)
plt.ylabel('True positive rate',fontsize = 14)
plt.legend(loc='lower right', fontsize = 14)
plt.suptitle('ROC curves for different models', fontsize=14)
plt.xlim([0,1])
plt.ylim([0,1])

### c)

In [ ]:
graphviz.Source(source = tree.export_graphviz(decisionTree))

In [ ]:
#The columns in order are: 'age', 'sex', 'income', 'married', 'children', 'car','save_acct', 'current_acct', 'mortgage', 10:14 is encoding for 'region'

We splitted on income first. The usual splitting attributes are: income, children, and age. Note that we are trying to predict PEP (whether the person bought a Personal Equity Plan after the last mailing). 

I think that these results show the important things to focus on if the bank is trying to get more people to purchase their PEP. Since this is a financial decision, I was not suprised to see "income" was the attribute being used the most as well as it being the first split in the tree. Number of children being the second most splitted attributes along with the fact that the marriage status attribute was also being used quite frequently combined together suggest that this is also something to look out for. Perharps people who have to support their family will look more into personal investment hence having a higher chance of purchasing PEP. The attribute age also being a common one might be suggesting an investing pattern (maybe old people tend to be more financially responsible and the fact that they actually have money to invest will lead to them purchasing more PEP?).